In [1]:
import numpy as np
import pandas as pd
import json
from scipy.spatial.distance import hamming

In [2]:
# make a function that has the playlist column and the song column
def make_songlist(x):
    # make a dictionary for the first playlist and its songs as the root of the final dataframe
    d = {"playlist": 0, "song": x[0]}
    df = pd.DataFrame(d)
    # use a for loop to append the rest of the playlists and songs into the root dataframe
    for n in range(1, len(x)):
        d = {"playlist": n, "song": x[n]}
        df2 = pd.DataFrame(d)
        df = df.append(df2, ignore_index = True)
    return df

In [28]:
# load the first 1000 spotify playlists
with open("D:/Users/jacky/Desktop/Data Science/Capstone Projects/Music_Recommendations/spotify/spotify_million_playlist_dataset/data/mpd.slice.0-999.json") as f:
    data = json.load(f)

# extract the song names in each playlist and store it in a list respectively
song_playlist = {}
for i, n in enumerate(data["playlists"]):
    song_playlist[i] = []
    for m in n["tracks"]:
        song_playlist[i].append(m["track_name"])

make_songlist(song_playlist).shape

(67503, 2)

In [4]:
# load the first P sets of playlists (each set contains 1000 playlists)

def load_files(P):

    for n in range(P):
        a = str(0 + 1000*n)
        b = str(999 + 1000*n)
        filepath = "D:/Users/jacky/Desktop/Data Science/Capstone Projects/Music_Recommendations/spotify/spotify_million_playlist_dataset/data/mpd.slice." + a + "-" + b + ".json"
        with open(filepath) as f:
            data = json.load(f)

# extract the songs from the original dictionary
        song_playlist = {}
        for i, j in enumerate(data["playlists"]):
            song_playlist[i+1000*n] = []
            for m in j["tracks"]:
                song_playlist[i+1000*n].append(m["track_name"])

# append each playlist's songs into a dataframe for further proces
        if n == 0:
            df_playlist_song = make_songlist(song_playlist)
        else:
            for a in range(len(song_playlist)):
                d = {"playlist": a + 1000*n, "song": song_playlist[a + 1000*n]}
                df2 = pd.DataFrame(d)
                df_playlist_song = df_playlist_song.append(df2, ignore_index = True)

    return df_playlist_song

In [5]:
# Load the first 10000 playlists into the variable
df_playlist_song2 = load_files(10)
df_playlist_song2 = df_playlist_song2.set_index("playlist")
df_playlist_song2.to_csv("playlist_song2.csv")

In [27]:
df_playlist_song2.shape

(664712, 1)

In [26]:
# by one hot encoding and groupby functions, a pivot table with playlists against songs is produced
# song_matrix = pd.get_dummies(df_playlist_song["song"]).groupby(["playlist"]).sum()
# song_matrix.to_csv("song_matrix.csv")

In [1]:
# convert all 0s to np.nan for hamming distance calculation
# song_matrix[song_matrix==0] = np.nan
# song_matrix.to_csv("song_matrix.csv")

In [28]:
"""
# create a function to obtain the K nearest neighbours by hamming distance calculation
def nearestneighbours(user, K):
    # obtain a list of playlists without the target user's playlist to avoid duplicates before hamming calculation
    allusers = pd.DataFrame(song_matrix.index)
    allusers = allusers[allusers["playlist"]!= user]

    # apply hamming distance between the target playlist and the rest of the playlists and return K nearest results
    allusers["distance"] = allusers["playlist"].apply(lambda x: hamming(song_matrix.loc[user], song_matrix.loc[x]))
    Knearest = allusers.sort_values(["distance"], ascending = True)["playlist"][:K]
    return Knearest

def topN(user, x, K, N):
    # obtain KNN through the function created before and store the target neighbours songs into a variable
    KNearest = nearestneighbours(user, K)
    NNsongs = df_playlist_song[df_playlist_song.index.isin(KNearest)]

    # filter the song names by eliminating songs which are already in the user's playlist
    top_songs = NNsongs["song"].value_counts()#.index
    user_songs = df_playlist_song[df_playlist_song.index == user]
    if user == "user":
        user_songs = x[:10]
    top_songs = top_songs[~top_songs.index.isin(user_songs)][:N]

    return top_songs

# function to let a new user get recommnedation
def new_recommendation(x):
    user_songs2 = x[:10]
    song_matrix.loc["user"] = np.nan
    for song in user_songs2:
        if song in song_matrix.columns:
            song_index = list(song_matrix.loc["user"].index).index(song)
            song_matrix.iloc[len(song_matrix)-1 ,song_index] = 1
            print(song, "is in the database")
    final = topN("user", x, 50, 100)
    return final

# evaluation function to measure the succesful hit-rate of the recommendation system based on user-input playlist
def evaluation_metrics(x):
    list_recom = new_recommendation(x)
    list_remaining_songs = x[10:]
    total_songs = len(set(list_remaining_songs).intersection(song_matrix.columns))
    matched_songs = len(list_recom.index.intersection(list_remaining_songs))
    print(list_recom.index)

    matched_rate = matched_songs/total_songs

    return matched_rate, total_songs, matched_songs
"""

In [20]:
# create a function to obtain the K nearest neighbours by hamming distance calculation
def nearestneighbours_god(user_songs2, K):
    # obtain a list of playlists without the target user's playlist to avoid duplicates before hamming calculation
    df_god = pd.DataFrame(set(df_playlist_song2.index))
    df_god = df_god.set_index(0)
    df_god.index.names = ["playlist"]
    df_god["hamming"] = 1

    # apply hamming distance between the target playlist and the rest of the playlists and return K nearest results
    for n in df_god.index:
        song_matrix = pd.get_dummies(df_playlist_song2[df_playlist_song2.index == n], prefix="", prefix_sep="").groupby(["playlist"]).sum()
        song_matrix.loc["user"] = np.nan
        for song in user_songs2:
            if song in song_matrix.columns:
                song_index = list(song_matrix.loc["user"].index).index(song)
                song_matrix.iloc[len(song_matrix)-1 ,song_index] = 1

        df_god.loc[n, "hamming"] = hamming(song_matrix.loc["user"], song_matrix.loc[n])

    df_god = df_god.sort_values(["hamming"], ascending=True)[:K]

    return df_god

def topN_god(user_songs2, K, N):
    # obtain KNN through the function created before and store the target neighbours songs into a variable
    KNearest = nearestneighbours_god(user_songs2, K).index
    NNsongs = df_playlist_song2[df_playlist_song2.index.isin(KNearest)]

    # filter the song names by eliminating songs which are already in the user's playlist
    top_songs = NNsongs["song"].value_counts()#.index
    user_songs = user_songs2[:20]
    top_songs = top_songs[~top_songs.index.isin(user_songs)][:N]

    return top_songs

# function to let a new user get recommnedation
def new_recommendation_god(user_songs):
    user_songs2 = user_songs[:20]
    for song in user_songs2:
        if song in list(df_playlist_song2["song"]):
            print(song, "is in the database")
    final = topN_god(user_songs2, 50, 100)
    return final

# evaluation function to measure the succesful hit-rate of the recommended song list based on user-input playlist
def evaluation_metrics_god(x):
    list_recom = new_recommendation_god(x)
    list_remaining_songs = x[20:]
    total_songs = len(set(list_remaining_songs).intersection(df_playlist_song2["song"]))
    matched_songs = len(list_recom.index.intersection(list_remaining_songs))
    print(list_recom.index)

    matched_rate = matched_songs/total_songs
    results = f"""Songs that appear in Spotify database: {total_songs}\nSongs that match our recommended songs: {matched_songs}\nMatching Rate: {round(matched_rate*100, 2)}%"""

    return results

In [8]:
user_playlist = ["Tik Tok","Need You Now","Hey, Soul Sister","CaliforniaGurls","OMG","Airplanes",
        "Love the Way You Lie","Bad Romance","Dynamite","Break Your Heart",
        "Nothin' on You","I Like It","BedRock","In My Head","Rude Boy",
        "Telephone","Teenage Dream","Just the Way You Are","Cooler Than Me","Imma Be",
        "Empire State of Mind","DJ Got Us Fallin' in Love","Billionaire"
        ,"Not Afraid","Replay","Sexy Bitch","Breakeven","Your Love Is My Drug",
        "I Gotta Feeling","Fireflies","Say Aah","Find Your Love","Alejandro"
        ,"Ridin' Solo","Just a Dream","How Low","Like a G6","Carry Out","Haven't Met You Yet",
        "Club Can't Handle Me","Down","Bulletproof","Whatcha Say","Baby","Whataya Want from Me"
        ,"Mine","Only Girl (In the World)","Live Like We're Dying","Hard","Young Forever","Blah Blah Blah"
        ,"Bottoms Up","Do You Remember","All the Right Moves","According to You","My Chick Bad",
        "You Belong with Me","Meet Me Halfway","Take It Off","Over","Animal","Misery","Magic","Paparazzi"
        ,"Tie Me Down","Your Love","Party in the U.S.A.","Deuces","3","Impossible","Forever"
        ,"Two Is Better Than One","My First Kiss","Already Gone","Rock That Body","Secrets"
        ,"Naturally","Un-Thinkable (I'm Ready)","All I Do Is Win","I Made It (Cash Money Heroes)"
        ,"Stuck Like Glue","Hey Daddy (Daddy's Home)","There Goes My Baby","Today Was a Fairytale"
        ,"Say Something","Sweet Dreams","Use Somebody","Undo It","Eenie Meenie","Right Above It"
        ,"The House That Built Me","If I Die Young","The Only Exception","American Honey"
        ,"King of Anything","Life After You","Smile","Teach Me How to Dougie","Try Sleeping with a Broken Heart"
        ,"Lover, Lover"]

In [46]:
#df_god = pd.DataFrame(set(df_playlist_song2.index))

In [21]:
print(evaluation_metrics_god(user_playlist))

Tik Tok is in the database
Need You Now is in the database
Hey, Soul Sister is in the database
OMG is in the database
Airplanes is in the database
Love the Way You Lie is in the database
Bad Romance is in the database
Dynamite is in the database
Break Your Heart is in the database
Nothin' on You is in the database
I Like It is in the database
BedRock is in the database
In My Head is in the database
Rude Boy is in the database
Telephone is in the database
Teenage Dream is in the database
Just the Way You Are is in the database
Cooler Than Me is in the database
Imma Be is in the database
Index(['Whatcha Say', 'Replay', 'Down', 'Kiss Me Thru The Phone',
       'Party In The U.S.A.', 'I'm Yours', 'TiK ToK',
       'Only Girl (In The World)', 'Last Friday Night (T.G.I.F.)', 'SexyBack',
       'Hey There Delilah', 'Grenade', 'Umbrella', 'I Gotta Feeling',
       'Nothin' On You (feat. Bruno Mars)', 'Beautiful Girls',
       'California Gurls - feat. Snoop Dogg',
       'Airplanes (feat. Hayl

In [24]:
user_playlist2 = ["Dance With the Devil","Moves","DEVASTATED","Back At It","No Role Modelz","Stronger","BUTTERFLY EFFECT","rockstar","Swimming Pools (Drank)","The Show Goes On","Can't Tell Me Nothing","XO Tour Llif","Chaos","Jocelyn Flores","Everybody","LOYALTY. FEAT. RIHANNA.","I Fall Apart","Everybody Dies In Their Nightmares","It's A Vibe Chainz","Lemon","Move Bitch","Heathens","Only","Chun-Li","APESHIT","Feelin' Myself","Chun-Li","Trini Dem Girls","Feeling Myself","Berzerk","Mockingbird","Just Lose It","Like Toy Soldiers","Superman","Without Me","Cleanin' Out My Closet","Stan","My Name Is","SICKO MODE","Jugg","Redstripe Rhapsody","Lalala","Devastated","NEW MAGIC WAND","Bodak Yellow","Seventeen","Anaconda","Bank Account","Look At Me!","Gucci Gang","Mask Off","Sad Nibba Hours","Savage Mode","Run This Town","Bad and Boujee (feat. Lil Uzi Vert)","Blue World","GOTTASADAE","Blue World","Craftmanship","No Type","LIL BIT","Yoshi City","Kyoto","The Box","Psycho!","THE SCOTTS","goosebumps","Ginseng Strip ","Red Bottom Sky","Do Not Disturb","Crush","Immortal","Tired of Me","Clap Clap","Get Out My Way","I'm Not Racist","POISON KLAN","World Wide Web","Bound ","Black Skinhead","This Is America","holy terrain","Riot","Wild for the Night","Versace (Remix)","Heartless","Amazing","Love Lockdown","BLACK PARADE","Antisocial (with Travis Scott)","Unforgettable","No Frauds","Get It","Bllionz","Flashing Lights","Who Dat Boy","Garbage","THE PLAN","WET","Shabba"]

In [25]:
new_recommendation_god(user_playlist2)

Dance With the Devil is in the database
Moves is in the database
DEVASTATED is in the database
Back At It is in the database
No Role Modelz is in the database
Stronger is in the database
rockstar is in the database
Swimming Pools (Drank) is in the database
The Show Goes On is in the database
Can't Tell Me Nothing is in the database
Chaos is in the database
Jocelyn Flores is in the database
Everybody is in the database
LOYALTY. FEAT. RIHANNA. is in the database
I Fall Apart is in the database
Everybody Dies In Their Nightmares is in the database
Lemon is in the database


Congratulations                   11
HUMBLE.                            9
XO TOUR Llif3                      8
Fuck Love (feat. Trippie Redd)     8
1-800-273-8255                     8
                                  ..
Weekend (feat. Miguel)             2
Catch Me Outside                   2
Money Longer                       2
Ill Mind of Hopsin 7               2
Perplexing Pegasus                 2
Name: song, Length: 100, dtype: int64

In [26]:
print(evaluation_metrics_god(user_playlist2))

Dance With the Devil is in the database
Moves is in the database
DEVASTATED is in the database
Back At It is in the database
No Role Modelz is in the database
Stronger is in the database
rockstar is in the database
Swimming Pools (Drank) is in the database
The Show Goes On is in the database
Can't Tell Me Nothing is in the database
Chaos is in the database
Jocelyn Flores is in the database
Everybody is in the database
LOYALTY. FEAT. RIHANNA. is in the database
I Fall Apart is in the database
Everybody Dies In Their Nightmares is in the database
Lemon is in the database
Index(['Congratulations', 'HUMBLE.', 'XO TOUR Llif3',
       'Fuck Love (feat. Trippie Redd)', '1-800-273-8255', 'Unforgettable',
       'Young Dumb & Broke', 'Bodak Yellow', 'Drowning (feat. Kodak Black)',
       'Mask Off', 'White Iverson', 'Slippery (feat. Gucci Mane)',
       'Gucci Gang', 'One Night', 'do re mi', 'Magnolia',
       'I Don't Fuck With You', 'Swang', 'Bounce Back', 'Go Flex', 'Caroline',
       'Wild 

In [51]:
"""
from scipy.spatial import distance
import numpy as np
distance.hamming([1, 0, 0, np.nan, np.nan], [0, 1, 0,  np.nan, np.nan])
"""

0.8